# Domain knowledge steps in


After the talks with Dmitry I decided that every nan-value should be replaced with corresponding BAU value. Let's then do that and try same clusterings as before:

In [2]:
from gradutil import *
import pandas as pd
from pyomo.opt import SolverFactory

In [3]:
revenue, carbon, deadwood, ha = init_boreal()
x_revenue = nan_to_bau(revenue)
x_carbon = nan_to_bau(carbon)
x_deadwood = nan_to_bau(deadwood)
x_ha = nan_to_bau(ha)

In [4]:
x = np.concatenate((x_revenue.values, x_carbon.values, x_deadwood.values, x_ha.values), axis=1)
x_norm = normalize(x)

In [5]:
opt = SolverFactory('glpk')
value_revenue, value_carbon, value_deadwood, value_ha = cNopt(x, x_norm, x, opt, nclust=10, seed=2)

In [6]:
value_revenue, value_carbon, value_deadwood, value_ha

In [7]:
tmp_x = np.concatenate((x_ha.values,x_ha.values,x_ha.values,x_ha.values), axis=1)
tmp = cNopt(tmp_x, normalize(tmp_x), tmp_x, opt, nclust=100, seed=4)

In [8]:
centers, xtoc, dist = cluster(x_ha.values, 50, seed=3)

In [9]:
%%time
weights = np.array([sum(xtoc==i) for i in range(len(centers))])
clustProblemHA = BorealWeightedProblem(centers,weights)
opt = SolverFactory('glpk')
resClustHA = opt.solve(clustProblemHA.model, False)

In [10]:
val_ha = model_to_real_values(x_ha.values, xtoc, clustProblemHA.model)
real_ha = real_solutions()['ha']
(val_ha-real_ha)/real_ha

So well, the Nan to BAU by itself didn't really help...